# Retrieval Application

## Vector Database (Vector DB)
Resources
- [How-to guides](https://python.langchain.com/v0.2/docs/how_to/#vector-stores)
  - [Vectorstores](https://python.langchain.com/v0.2/docs/integrations/vectorstores/): A vector store that stores embedded data and performs similarity search.
    1. [Elasticsearch](https://python.langchain.com/v0.2/docs/integrations/vectorstores/elasticsearch/)
    2. [Milvus](https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/)
    3. [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Environment Setup

Sources  
- [langchain-chroma](https://pypi.org/project/langchain-chroma/)

In [2]:
from importlib.metadata import version
# !pip install langchain
# Select langchain to 0.1.20
try:
    print('langchain package version',version('langchain'))
    assert version('langchain') == '0.1.20'
except:
    !pip install langchain==0.1.20

# !pip install --upgrade langchain
# print('langchain package version',version('langchain'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [3]:
#!pip install -qU langchain-huggingface
# Select langchain-huggingface to 0.0.3
try:
    print('langchain-huggingface package version',version('langchain-huggingface'))
    assert version('langchain-huggingface') =='0.0.3'#'0.2.11'
except:
    !pip install langchain-huggingface==0.0.3
    #0.2.11 (if any)

# !pip install -qU langchain-huggingface
# print('langchain-huggingface package version',version('langchain-huggingface'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.6 MB/s eta 0:00:00


In [4]:
# Select langchain-chroma to 0.1.3
try:
    print('langchain_chroma package version',version('langchain_chroma'))
    assert version('langchain_chroma') == '0.1.3'
except:
    !pip install langchain_chroma==0.1.3

# !pip install -qU langchain_chroma==0.1.3
# print('langchain_chroma package version',version('langchain_chroma'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.4 MB/s eta 0:00:

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import os
#check current directory
os.getcwd()

Data  sample_data


In [7]:
# check folders / files in current directory
!dir

Data  sample_data


In [8]:
import langchain_chroma
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Import Data
- Data source: [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings)

**Features of Use**
- id:
  1. job_id  

- Main data  

  **Texts**  
  1. _description_
  1. _skills_desc_  

  **Vector**  
  1. title
  2. _description_
  3. _skills_desc_  
  
- Meta data:  
  1. title
  1. location
  1. min_salary
  1. pay_period
  1. job_posting_url
  
  


In [30]:
df = pd.read_csv('./Data/archive/postings.csv', delimiter=',') #, engine="python"

print(f"The variables (features) of the data:\n{df.columns}")
df.head(5)

The variables (features) of the data:
Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [31]:
df

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,3906267117,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,195000.0,YEARLY,"Walnut Creek, CA",56120.0,1.0,NaN,...,NaN,1.713571e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,94595.0,6013.0
123845,3906267126,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,NaN,NaN,United States,1124131.0,3.0,NaN,...,NaN,1.713572e+12,www.pinterestcareers.com,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN
123846,3906267131,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,NaN,NaN,"Spokane, WA",90552133.0,3.0,NaN,...,NaN,1.713572e+12,epsoperations.bamboohr.com,0,FULL_TIME,NaN,NaN,NaN,99201.0,53063.0
123847,3906267195,Trelleborg Applied Technologies,Business Development Manager,The Business Development Manager is a 'hunter'...,NaN,NaN,"Texas, United States",2793699.0,4.0,NaN,...,NaN,1.713573e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df['job_post'] = df['description'].astype('str') + df['skills_desc'].astype('str')
# df[['description', 'skills_desc', 'job_post']].head()

In [32]:
df['job_post'] = np.where(df['skills_desc'].isna(), df['description'].astype('str'), df['skills_desc'].astype('str'))
df[['description', 'skills_desc', 'job_post']].head()

,description,skills_desc,job_post
0,Job descriptionA leading real estate firm in N...,Requirements: \n\nWe are seeking a College or ...,Requirements: \n\nWe are seeking a College or ...
1,"At Aspen Therapy and Wellness , we are committ...",NaN,"At Aspen Therapy and Wellness , we are committ..."
2,The National Exemplar is accepting application...,We are currently accepting resumes for FOH - A...,We are currently accepting resumes for FOH - A...
3,Senior Associate Attorney - Elder Law / Trusts...,This position requires a baseline understandin...,This position requires a baseline understandin...
4,Looking for HVAC service tech with experience ...,NaN,Looking for HVAC service tech with experience ...


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

## Select Data
**Purpose**:
- Insert data samples into Vector Database (VectorDB), working as knowledge base

**Note**:
- For demonstration, I pick 50% of data related to a specific job tiltes, and 50% other random job titles

In [34]:
# define yourself

## job title you're interested
keywords = ['data science', 'data scientist', 'data analyst']
# keywords = ['data analytics', 'data scientist', 'hr analytics', 'hr data analyst']

## Number of relevant / irrelevant samples
N = 400

In [35]:
condition = df['title'].str.contains('|'.join(keywords), case=False, na=False)
df_ds = df[condition]
# 'case=False' makes the search case-insensitive
# 'na=False' ensures that NaN values are not considered in the search

N_ds = min(len(df_ds), N)
df_ds = df_ds.sample(n=N_ds)
print(f"There're {N_ds} samples searched according to keywords.")
df_ds.head(10)

There're 400 samples searched according to keywords.


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,job_post
85406,3904366169,Alight Solutions,Senior Data Scientist - Virtual,"Our story \n\nAt Alight, we believe a company’...",162600.0,YEARLY,"Chicago, IL",21528485.0,159.0,NaN,...,1.713390e+12,careers.alight.com,0,FULL_TIME,USD,BASE_SALARY,132500.0,60601.0,17031.0,"Our story \n\nAt Alight, we believe a company’..."
97383,3904957834,Coders Data,Data Analyst,Actively participates in projects in assigned ...,NaN,NaN,New York City Metropolitan Area,14508644.0,10.0,NaN,...,1.713455e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,Actively participates in projects in assigned ...
63690,3902351871,Select Source International,Data Analyst III (Onsite - Only W2/No C2C),Job Title: Data Analyst III (Onsite)Location: ...,NaN,NaN,"Phoenix, AZ",77097.0,8.0,NaN,...,1.713537e+12,NaN,0,CONTRACT,NaN,NaN,NaN,85003.0,4013.0,Job Title: Data Analyst III (Onsite)Location: ...
77045,3903477932,MANN+HUMMEL,Catalog Data Analyst - Heavy Duty,Catalog Data Analyst – Heavy-DutyLocation: Gas...,NaN,NaN,"Gastonia, NC",259898.0,12.0,NaN,...,1.713462e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,28052.0,37071.0,Catalog Data Analyst – Heavy-DutyLocation: Gas...
50803,3901376493,DataAnnotation,Data Analyst,DataAnnotation is committed to creating qualit...,NaN,NaN,Greater Phoenix Area,96139831.0,107.0,NaN,...,1.713458e+12,NaN,0,CONTRACT,NaN,NaN,NaN,NaN,NaN,DataAnnotation is committed to creating qualit...
75003,3903444713,4A IT Services LLC,Sr Data scientist,Job Title: Data ScientistLocation: San Francis...,NaN,NaN,San Francisco Bay Area,68257427.0,91.0,NaN,...,1.713454e+12,NaN,0,CONTRACT,NaN,NaN,NaN,NaN,NaN,Job Title: Data ScientistLocation: San Francis...
50488,3901373790,SNVA Technologies,Data Analyst,\n\nTitle: Data Analyst (Onsite)\n\nWork autho...,NaN,NaN,"Juno Beach, FL",14439628.0,16.0,NaN,...,1.713459e+12,NaN,0,CONTRACT,NaN,NaN,NaN,34217.0,12081.0,\n\nTitle: Data Analyst (Onsite)\n\nWork autho...
61934,3901998589,JPMorgan Chase & Co.,"VP / Client, Finance & Strategy Data Scientist...","Job Description\n\nVP / Client, Finance & Stra...",NaN,NaN,"San Francisco, CA",1068.0,4.0,NaN,...,1.713501e+12,JPMorganChase.contacthr.com,0,FULL_TIME,NaN,NaN,NaN,94101.0,6075.0,"Job Description\n\nVP / Client, Finance & Stra..."
97178,3904955466,DataAnnotation,Data Analyst,DataAnnotation is committed to creating qualit...,NaN,NaN,Denver Metropolitan Area,96139831.0,144.0,NaN,...,1.713454e+12,NaN,0,CONTRACT,NaN,NaN,NaN,NaN,NaN,DataAnnotation is committed to creating qualit...
24165,3890896231,MatchPoint,Lead Data Analyst,"Job Title: Lead Analyst, Advanced AnalyticsDut...",NaN,NaN,San Francisco Bay Area,223659.0,14.0,NaN,...,1.712857e+12,NaN,0,CONTRACT,NaN,NaN,NaN,NaN,NaN,"Job Title: Lead Analyst, Advanced AnalyticsDut..."


In [36]:
df_others = df[~condition].sample(n=N)
print(f"There're {len(df_others)} samples searched not contained in keywords.")
df_others.head(10)

There're 400 samples searched not contained in keywords.


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,job_post
49732,3901359191,Kansas Appleseed Center for Law and Justice,Just Campaign Director,Kansas Appleseed is seeking application for ou...,75000.0,YEARLY,"Lawrence, KS",10169731.0,32.0,NaN,...,1.713454e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,72500.0,66044.0,20045.0,Kansas Appleseed is seeking application for ou...
109406,3905359928,Pella Windows & Doors Mountain West,Trade Sales Consultant,Job Details\n\nJob Location\n\nScottsdale Show...,NaN,NaN,"Scottsdale, AZ",3880846.0,4.0,NaN,...,1.713493e+12,www.paycomonline.net,0,FULL_TIME,NaN,NaN,NaN,85250.0,NaN,Job Details\n\nJob Location\n\nScottsdale Show...
46481,3900966305,ComTec Information Systems (IT),Help Desk Specialist,Title: Help Desk/Technical Support EngLocation...,NaN,NaN,"Murrysville, PA",1745669.0,8.0,NaN,...,1.713393e+12,NaN,0,CONTRACT,NaN,NaN,NaN,15668.0,42129.0,Title: Help Desk/Technical Support EngLocation...
25444,3891074898,SGS,3rd Party Pre-Shipment Inspectors (Contract),"We are SGS – the world's leading testing, insp...",NaN,NaN,"Buffalo, NY",4126.0,3.0,NaN,...,1.712864e+12,jobs.smartrecruiters.com,0,CONTRACT,NaN,NaN,NaN,14201.0,36029.0,"We are SGS – the world's leading testing, insp..."
107241,3905331976,PAM Health Specialty Hospital of Heritage Valley,Patient Care Technician (PCT/CNA) - FT Nights ...,Patient Care Technician\n\nThe Patient Care Te...,NaN,NaN,"Beaver, PA",96209108.0,5.0,NaN,...,1.713481e+12,recruiting.ultipro.com,0,FULL_TIME,NaN,NaN,NaN,15009.0,42007.0,Patient Care Technician\n\nThe Patient Care Te...
66635,3902747737,Paychex,PEO Implementation Benefit Consultant - locate...,Overview\n\nActs as project manager on behalf ...,NaN,NaN,"Piscataway, NJ",4255.0,11.0,NaN,...,1.713392e+12,careers.paychex.com,0,FULL_TIME,NaN,NaN,NaN,8854.0,34023.0,Overview\n\nActs as project manager on behalf ...
121645,3906248739,Integrated DNA Technologies,Sr. Total Rewards Manager - REMOTE,Integrated DNA Technologies (IDT) is the leadi...,125000.0,YEARLY,"New York, United States",21041.0,4.0,NaN,...,1.713566e+12,jobs.danaher.com,0,FULL_TIME,USD,BASE_SALARY,117500.0,NaN,NaN,Integrated DNA Technologies (IDT) is the leadi...
91815,3904565220,NaN,Logistics Coordinator,Role DescriptionThis is a full-time remote rol...,NaN,NaN,United States,NaN,149.0,NaN,...,1.713551e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN,Role DescriptionThis is a full-time remote rol...
45521,3900958973,City of Hope,Systems Engineer - Storage Remote,"Join the transformative team at City of Hope, ...",64.0,HOURLY,"Duarte, CA",163079.0,17.0,NaN,...,1.713392e+12,www.cityofhopejobs.org,0,FULL_TIME,USD,BASE_SALARY,109200.0,91008.0,NaN,"Join the transformative team at City of Hope, ..."
61019,3901985223,MiQ,Senior Account Executive,Role: Senior Account Executive\n\nLocation: Ne...,140000.0,YEARLY,"New York, NY",2883353.0,5.0,NaN,...,1.713494e+12,www.wearemiq.com,0,FULL_TIME,USD,BASE_SALARY,132500.0,10001.0,36061.0,Role: Senior Account Executive\n\nLocation: Ne...


In [37]:
df_select = pd.concat([df_ds, df_others])

print(f"There're totally {len(df_select)} samples for inserting VectorDB.")

There're totally 800 samples for inserting VectorDB.


# Create VectorDB

## Create Indexer: Construct the Container (aka. collection) for VectorDB

In [41]:
encoder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [42]:
encoder

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [43]:
import chromadb
collection_name = "collection_postings"

persistent_client = chromadb.PersistentClient()
# print("check all functions/attribute for Chroma:\n", dir(persistent_client))



In [44]:
if not persistent_client.list_collections():
    print('empty')

empty


In [45]:
import chromadb
collection_name = "collection_postings"

persistent_client = chromadb.PersistentClient()
# print("check all functions/attribute for Chroma:\n", dir(persistent_client))

if not persistent_client.list_collections():
    print(f"{collection_name} not in collection (DB) yet!")
else:
    if collection_name in persistent_client.list_collections()[0].name:
        print(f"{collection_name} is already in collection and would be deleted!")
        persistent_client.delete_collection(collection_name)

print(f"Create collection: {collection_name}!")
vector_store = Chroma(
    client=persistent_client,
    collection_name=collection_name,
    embedding_function=encoder,
    persist_directory="./chroma_langchain_db",  #save data locally, remove if not neccesary
)

#print("check all functions/attribute for Chroma:\n", dir(vector_store))

# if collection_name in vector_store.list_collections():
#     vector_store.delete_collection(collection_name)


collection_postings not in collection (DB) yet!
Create collection: collection_postings!


## Indexing: Insert Data into VectorDB


In [46]:
from langchain_core.documents import Document
ids = []
documents = []
for index, row in df_select.iterrows():
    id_current = str(index)
    ids.append(id_current)

    min_salary = row['min_salary']
    if pd.isna(min_salary):
        min_salary = 0  # or any default value you prefer

    document_current = Document(
        page_content=row['job_post'],
        metadata={"title": row['title'],
                  "location": row['location'],
                  "min_salary": min_salary,
                  "pay_period": row['pay_period'],
                  "job_posting_url": row['job_posting_url'],
                 },
        id=row['job_id'],
    )
    documents.append(document_current)
print(f"There are {len(documents)}.\n")
print(f'Example document content:\n{document_current}')

There are 800.

Example document content:
page_content='Position Title: Brow Artist (Licensed)\nMission: Perform eyebrow tweezing based on our unique protocols and be able to prescribe the correct eyebrow shaping and home care for a diverse clientele. Maintain a 70% occupancy rate and grow our brow services by 200% by the end of 2024.\nOutcomes:Grow revenue specifically coming from brow tweezing services to $8,000 per month by the 6 month mark.Write up client notes inside their Acuity profile after every appointment.Take continuing education classes provided by the company which may include online programs, in-person events and/or on-site training.\nActivities:Clean the brow station to standard in between every client and our deep cleaning protocol at the end of every day.Restock your station at the end of every shift.Take, edit and upload Before & After client photos.Write up client notes inside their Acuity profile after every appointment.Keep your workstation clean, organized, stock

In [47]:
vector_store.add_documents(documents=documents, ids=ids)

['85406',
 '97383',
 '63690',
 '77045',
 '50803',
 '75003',
 '50488',
 '61934',
 '97178',
 '24165',
 '51643',
 '106560',
 '68028',
 '55257',
 '2658',
 '65384',
 '10243',
 '56756',
 '58363',
 '10765',
 '98937',
 '79848',
 '14019',
 '27191',
 '23949',
 '5826',
 '91845',
 '45443',
 '13246',
 '93175',
 '9410',
 '60859',
 '6531',
 '75010',
 '72049',
 '68796',
 '38342',
 '74192',
 '75723',
 '116000',
 '97067',
 '54286',
 '99668',
 '107150',
 '34130',
 '31530',
 '70329',
 '104785',
 '46543',
 '17573',
 '21150',
 '61704',
 '94064',
 '76625',
 '29213',
 '55869',
 '10263',
 '27547',
 '77887',
 '27471',
 '3730',
 '22681',
 '61620',
 '74972',
 '41024',
 '95928',
 '68226',
 '22377',
 '53326',
 '96430',
 '14627',
 '2887',
 '28496',
 '28712',
 '117781',
 '97608',
 '38826',
 '9582',
 '98067',
 '97499',
 '114451',
 '55037',
 '50861',
 '38176',
 '51339',
 '108413',
 '92604',
 '56732',
 '121441',
 '114360',
 '35843',
 '116213',
 '51937',
 '122094',
 '94856',
 '49843',
 '91281',
 '52380',
 '77914',
 '4764

## Search: VectorDB as Retriever

### Vector Search (Similarity Search)
According to the query, find k=5 best possible job posts

In [48]:
query = "I am a passionate job seeker with a strong desire to embark on a career in data science. Having recently graduated with a degree in Computer Science, I have honed my skills in Python programming and developed a deep interest in machine learning. During my studies, I immersed myself in various projects that allowed me to apply these skills, from building predictive models to analyzing large datasets. My journey into data science has been driven by a fascination with uncovering hidden patterns in data and using these insights to solve real-world problems. I am now seeking an entry-level data scientist role where I can leverage my Python expertise and enthusiasm for machine learning to contribute to a dynamic team, learn from experienced professionals, and continue to grow my skills in this exciting field."
print(query)

I am a passionate job seeker with a strong desire to embark on a career in data science. Having recently graduated with a degree in Computer Science, I have honed my skills in Python programming and developed a deep interest in machine learning. During my studies, I immersed myself in various projects that allowed me to apply these skills, from building predictive models to analyzing large datasets. My journey into data science has been driven by a fascination with uncovering hidden patterns in data and using these insights to solve real-world problems. I am now seeking an entry-level data scientist role where I can leverage my Python expertise and enthusiasm for machine learning to contribute to a dynamic team, learn from experienced professionals, and continue to grow my skills in this exciting field.


In [50]:
results = vector_store.similarity_search_with_score(
    query , k=10,
)
i =1
for res, score in results:
    print(f"* [{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    i +=1

* [1][SIM=0.397912] Data Science Analyst - Full Time
---------------------
           Role: Data Science Analyst Type: Full-Time Job 
Ideally a US Citizen or Green Card holder within the DMV area preferably.
Position Summary:We are looking for an experienced Senior Data Science Analyst to lead our data science initiatives, leveraging expertise in Large Language Models (LLMs) and Python todrive data-driven decision-making. This role requires a deep understanding of data analytics, machine learning, and the ability to mentor junior analysts.
Key Responsibilities:• Design and implement advanced data models and algorithms using LLMs and Python.• Conduct complex data analysis to extract actionable insights and inform strategic decisions.• Develop/oversee the creation of sophisticated data visualizations and analytical reports.• Lead cross-functional projects and collaborate with stakeholders to achieve data-centricsolutions.• Mentor junior data science team members and contribute to their p

### Vector Search with Filtering 1
According to the query, find k=5 best possible job posts which have minimum salary greater than (gt) 100000

In [51]:
results = vector_store.similarity_search_with_score(
    query , k=3, filter={"min_salary": {"$gt": 100000}}
)# perator: $gt, $gte, $lt, $lte, $ne, $eq, $in, $nin
i = 1
for res, score in results:
    print(f"* [{i}][SIM={score:3f}] {res.metadata['title']}\n---------------------\n \
          {res.page_content} \n--------------------\n \
           [{res.metadata}]\n\n")
    i +=1

* [1][SIM=0.397912] Data Science Analyst - Full Time
---------------------
           Role: Data Science Analyst Type: Full-Time Job 
Ideally a US Citizen or Green Card holder within the DMV area preferably.
Position Summary:We are looking for an experienced Senior Data Science Analyst to lead our data science initiatives, leveraging expertise in Large Language Models (LLMs) and Python todrive data-driven decision-making. This role requires a deep understanding of data analytics, machine learning, and the ability to mentor junior analysts.
Key Responsibilities:• Design and implement advanced data models and algorithms using LLMs and Python.• Conduct complex data analysis to extract actionable insights and inform strategic decisions.• Develop/oversee the creation of sophisticated data visualizations and analytical reports.• Lead cross-functional projects and collaborate with stakeholders to achieve data-centricsolutions.• Mentor junior data science team members and contribute to their p